In [277]:
import os
import pandas as pd

In [278]:
dataframe_customers_dirty = pd.read_csv(
    os.path.join(
        "..",
        "data",
        "raw",
        "customers_dirty.csv"
    ),
    sep=","
)

dataframe_customers_dirty_2 = pd.read_csv(
    os.path.join(
        "..",
        "data",
        "raw",
        "customers_dirty2.csv"
    ),
    sep=","
)

dataframe_customers_dirty_3 = pd.read_csv(
    os.path.join(
        "..",
        "data",
        "raw",
        "customers_dirty3.csv"
    ),
    sep=","
)

In [279]:
# Columns types correction

# Dataframe 1
dataframe_customers_dirty["age"] = dataframe_customers_dirty["age"].fillna(0).astype(int)

dataframe_customers_dirty["signup_date"] = dataframe_customers_dirty["signup_date"].replace("not_a_date", pd.NaT)
dataframe_customers_dirty["signup_date"] = dataframe_customers_dirty["signup_date"].astype("datetime64[ns]")

# Dataframe 2
dataframe_customers_dirty_2["age"] = dataframe_customers_dirty_2["age"].fillna(0).astype(int)

dataframe_customers_dirty_2["signup_date"] = dataframe_customers_dirty_2["signup_date"].replace("invalid_date", pd.NaT)
dataframe_customers_dirty_2["signup_date"] = dataframe_customers_dirty_2["signup_date"].replace("2025-02-29", "2025-02-28")
dataframe_customers_dirty_2["signup_date"] = dataframe_customers_dirty_2["signup_date"].astype("datetime64[ns]")

In [280]:
# Add missing @ in email addresses

# Dataframe 1
dataframe_customers_dirty["email"] = dataframe_customers_dirty["email"].apply(
    lambda check: check if "@" in check else check.replace("example.com", "@example.com")
)

# Dataframe 2
dataframe_customers_dirty_2["email"] = dataframe_customers_dirty_2["email"].apply(
    lambda check: check if "@" in check else check.replace("example.com", "@example.com")
)

dataframe_customers_dirty_2["prenom"] = dataframe_customers_dirty_2["full_name"].str.split().str[0]
dataframe_customers_dirty_2["nom"] = dataframe_customers_dirty_2["full_name"].str.split().str[1]
domain = dataframe_customers_dirty_2["email"].str.split("@").str[1]

mask_email = dataframe_customers_dirty_2["email"].str.contains(r"\.[a-zA-Z]@")
dataframe_customers_dirty_2.loc[mask_email, "email"] = (
    dataframe_customers_dirty_2.loc[mask_email, "prenom"].str.lower()
    + "."
    + dataframe_customers_dirty_2.loc[mask_email, "nom"].str.lower()
    + "@"
    + domain
)

dataframe_customers_dirty_2.drop(columns=[
    "prenom",
    "nom"
], inplace=True)

In [281]:
# Fill missing signup dates with a median date

# Dataframe 1
median_date = dataframe_customers_dirty["signup_date"].median()
dataframe_customers_dirty["signup_date"] = dataframe_customers_dirty["signup_date"].fillna(median_date)

# Dataframe 2
median_date = dataframe_customers_dirty_2["signup_date"].median()
dataframe_customers_dirty_2["signup_date"] = dataframe_customers_dirty_2["signup_date"].fillna(median_date)

In [282]:
# Fix country names to uppercase

# Dataframe 1
dataframe_customers_dirty["country"] = dataframe_customers_dirty["country"].str.upper()

# Dataframe 2
dataframe_customers_dirty_2["country"] = dataframe_customers_dirty_2["country"].replace("France", "FR")
dataframe_customers_dirty_2["country"] = dataframe_customers_dirty_2["country"].str.upper()

In [283]:
# Check age values between 16 and 99

# Dataframe 1
dataframe_customers_dirty["age"] = dataframe_customers_dirty["age"].apply(
    lambda age: age if 16 <= age <= 99 else 16
)

# Dataframe 2
dataframe_customers_dirty_2["age"] = dataframe_customers_dirty_2["age"].apply(
    lambda age: age if 16 <= age <= 99 else 16
)

In [284]:
# Set negative last purchase amounts to zero

# Dataframe 1
dataframe_customers_dirty["last_purchase_amount"] = dataframe_customers_dirty["last_purchase_amount"].apply(
    lambda price: price if price >= 0.0 else 0.0
)

# Dataframe 2
dataframe_customers_dirty_2["last_purchase_amount"] = dataframe_customers_dirty_2["last_purchase_amount"].apply(
    lambda price: price if price >= 0.0 else 0.0
)

In [285]:
dataframe_customers_dirty.to_csv(
    os.path.join(
        "..",
        "data",
        "processed",
        "customers_cleaned.csv"
    ),
    index=False
)

dataframe_customers_dirty_2.to_csv(
    os.path.join(
        "..",
        "data",
        "processed",
        "customers_cleaned2.csv"
    ),
    index=False
)